In [ ]:
#@title
from google.colab import drive
! [ -e /content ] && pip install -Uqq fastbook
! pip install torch-lr-finder
! pip install -U sentence-transformers
import fastbook
fastbook.setup_book()
from fastai.tabular.all import *

In [134]:
#@title
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from fastbook import *
import gensim.downloader as gs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import random as r
from torch_lr_finder import LRFinder
from scipy import stats
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import sklearn.metrics as metrics
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sentence_transformers import SentenceTransformer

In [135]:
embed_model = SentenceTransformer('whaleloops/phrase-bert') # This is a phrase-bert model we will be using
embed_length = len(embed_model.encode("the mitochondria is the powerhouse of the cell"))

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/670 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [136]:
#@title
dumb_british_spellings = {"vapour":"vapor", "honour":"honor", "organisation":"organization", "harbour":"harbor", "odour":"odor", "centre":"center", "analyse":"analyze", "theatre":"theater", "colour":"color", "rumour":"rumor", "behaviour":"behavior", "armour":"armor", "grey":"gray", 
"mould":"mold", "neighbour":"neighbor", "axe":"ax", "moustache":"mustache", "plough":"plow", "mandarine":"mandarin"}
obscure_words = {"adj":"", "gipsy":"", "albumen":"", "campanula":"", "boletus edulis":"penny bun fungus", "ursus":"", "swearword":"swear word", 'adj.':"", "coleus":"", 'n.':"", 'ОК':"ok", "typha":"cattail", "pacifica":"peaceful", "mustella":"", "smail":"", "one’s":"", "spurflower":"perennial plant", "sabre":"","equus":"", "etc.":"", "ciconia":"",  "aër":"", "panthera":"panther", "еrinaceus":"", "e.g.":"", "centaurea":"thistle", "moschiferus":"", "apterus":"", "pyrrhocoris":"", "smn.":"", "pritchardia":"", "100":"one hundred", "sabrefish":"sabre carp", "putorius":"", "adv.":"", "petromyzontidae":"", "botaurus":"", "standart":"standard", "leccinum":"", "sg.":"", "gemini":"Gemini", "tabanidae":"", "anagallis":"", "decorticate":"stiff", "albugo":"", "frangula":"", "sciurus":"", "scrofa":"", "relig.":"", "headstream":"head stream", "solanum":"", "anguilla":"", "anat.":"", "nectarinia":"", "ipomoea":"", "repaire":"repair", "vaccinium":"", "smth":"", "smth.":"", "bubo":"", "deflorate":"remove flowers", "tr.":"", "traveller":"traveler", "bubalis":"", "marmorata":"", "furuncul":"", "caballus":"", "microchiroptera":"", "urtica":"", "plumbum":"", "biol.":"", "intr.":"", "bubalus":"", "columba":"", "cucurbita":"", "goldcrest":"small bird", "melongena":"", "picea":"", "arvensis":"", "moschus":"", "psidium":"", "radiointerference":"radio interference", "owre":"", "ricinus":"", "capricorn":"goat zodiac sign", "mustela":"", "pandion":"", "adj.of":"", "nomadize":"become nomadic", "smb.":"", "kneepit":"knee pit", "num.":"", "pl.":"", "extortioner":"extortion doer", "enculturate":"assimilate", "asquint":"squint", "uliginosum":"", "heteroptera":"", "ок":"ok", "abies":"fir", "stratiotes":"", "fiddlestick":"violin bow", "scabrum":"", "grus":"bird", "acarina":"", "guajava":"", "bitterling":"freshwater fish", "lycopersicum":"", "lutra":"otter", "plectranthus":"", "macereed":"mace reed", "24":"twenty four", "acris":"", "rotundifolius":"common weed", "gutturalis":"", "oxyeleotris":"", "geometrid":"", "citrullus":"", "lepus":"", "motacilla":"", "crake":"bird", "haliaёtus":"", "glasswort":"herb", "quinsy":"throat abscess", "shoulderblade":"shoulder blade", "spearthrower":"spear thrower", "ridgepole":"ridge pole", "pimpleface":"pimple face", "tumpline":"backpack", "cushma":"clothing", "curassow":"tropical bird", "banisterium":"plant", "paca":"rodent", "netbag":"net bag", "muntjacs":"barking deer"}
def clean(s):
  s = str(s)
  s = s.replace("(intr.)", "")
  #s = s.replace("\’s", "")
  for i in ",:":
    s = s.replace(i, " or")
  s = s.replace("/", " or ")
  for i in "()\t\n<>?":
    s = s.replace(i, "")
  for i in "-":
    s = s.replace(i, " ")
  should_be_upper = {}
  for w in s.split(" "):
    if w != "" and w[0].isupper():
      should_be_upper = {w.lower():w.capitalize()}
  s = s.lower()
  for i in dumb_british_spellings:
    s = s.replace(i, dumb_british_spellings[i])
  for i in obscure_words:
    s = s.replace(i, obscure_words[i])
  for i in should_be_upper:
    s = s.replace(i, should_be_upper[i])
  if len(s) > 2 and s[:3] == " or":
    s = s[3:]
  if "vs." in s:
    ind = s.find("vs.")
    s = s[:ind]
  s = s.replace(".","")
  return s

# Converts a number or array to a tensor of floats
def toTensor(arr):
  return torch.tensor(arr, dtype = torch.float32) #Function to easily convert arrays to tensors

In [137]:
#@title
NUM_LANGUAGE = 3156 # This is the number of languages used in the CLICS database
df_raw = pd.read_csv("gdrive/MyDrive/FUSRP/filtered_dat_sem_shift.csv") 
df = df_raw.copy()
df = df.iloc[[i for i in range(len(df)) if not "<" in df.iloc[i]["meaning2"]]]
df = df.reset_index().drop(["Unnamed: 0", "realizations", "index"], axis=1)
meaning1_clean = df["meaning1"].apply(clean)
meaning2_clean = df["meaning2"].apply(clean)
df.insert(2, "meaning1_clean", meaning1_clean)
df.insert(3, "meaning2_clean", meaning2_clean)

In [138]:
#@title
# Here we're creating a dictionary that we could turn into a dataframe, although in practice we won't until we convert everything to embeddings. 
# For each pair in datasemshift, we write it in the correct order with probability 1/2 and the incorrect order otherwise
# Then we will try to create models that can classify whether a pair is in the corerct order or not
# Essentially our model is predicting which direction the shift goes in
reversal_dic = {"meaning1":[], "meaning2":[], "reverse":[]}
for i in range(len(df)):
  if r.random() < 0.5:
    reversal_dic["meaning1"].append(df.iloc[i]["meaning1"])
    reversal_dic["meaning2"].append(df.iloc[i]["meaning2"])
    reversal_dic["reverse"].append(False)
  else:
    reversal_dic["meaning1"].append(df.iloc[i]["meaning2"])
    reversal_dic["meaning2"].append(df.iloc[i]["meaning1"])
    reversal_dic["reverse"].append(True)

In [ ]:
#@title
senses = list(set(list(df["meaning1"])+list(df["meaning2"]))) # List of all senses appearing in datsemshift
sense_indices = {senses[i]:i for i in range(len(senses))} # Dictionary that gives a sense's index in senses
vec_dic = {s:embed_model.encode(s) for s in senses} # Dictionary that gives a sense's phrase-BERT embedding

In [ ]:
#@title
# To create a dataframe used for our prediction models, for each pair of (possibly reversed) sense, we 
# Subtract their phrase bert embeddings, then train on these
# df_embeds is the dataframe of these subtracted embeddings

subtract_dic = {i:[vec_dic[reversal_dic["meaning2"][j]][i]-vec_dic[reversal_dic["meaning1"][j]][i] for j in range(len(df))] for i in range(embed_length)}
subtract_dic["reverse"] =reversal_dic["reverse"]
df_embeds = pd.DataFrame.from_dict(subtract_dic)

In [ ]:
 #@title
 # Dataframes of test and training data

train, test = train_test_split(df_embeds, test_size = 0.2)
X = df_embeds.drop("reverse", axis=1)
y = df_embeds["reverse"].to_frame()
X_train = train.drop("reverse", axis=1)
y_train = train["reverse"].to_frame()
X_test = test.drop("reverse", axis=1)
y_test = test["reverse"].to_frame()

In [ ]:
#@title
# Logistic regression model

logreg = LogisticRegression(max_iter = 10000)
logreg.fit(X_train, y_train)
print("Logistic regression accuracy: ", metrics.accuracy_score(logreg.predict(X_test), y_test))

In [ ]:
#@title
# (Untuned) random forest model

forest = RandomForestClassifier(max_depth = 10, n_estimators = 500, random_state = 0)
forest.fit(X_train, y_train)
print(metrics.accuracy_score(forest.predict(X_test), y_test))

In [ ]:
#@title
# (Untuned) XGBoost model

xgb = XGBClassifier(random_state = 0)
xgb.fit(X_train, y_train)
print(metrics.accuracy_score(xgb.predict(X_test), y_test))

In [ ]:
#@title
# Here we're setting up a neural network to test as well. 
# We convert the labels to floats as we will view these as probabilities of being in "TRUE" or "FALSE" classes

df_embeds['reverse'] = np.array(df_embeds['reverse'], dtype = np.float32) 
dep_var = "reverse"
inds = range(len(df_embeds))
splits = train_test_split(inds, test_size = 0.2)

In [ ]:
#@title
cont_nn,cat_nn = cont_cat_split(df_embeds, max_card=20, dep_var=dep_var) # Splits in continuous and catagorical data

In [ ]:
#@title
procs_nn = [Categorify, FillMissing]
to_nn = TabularPandas(df_embeds, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var)
dls = to_nn.dataloaders(32)

In [ ]:
#@title
n_in = len(dls.one_batch()[0][1]) # Our neural network
n_out = 1
model = nn.Sequential(
    nn.Linear(n_in, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, n_out),
    nn.ReLU(),
    nn.Sigmoid()
)

In [ ]:
#@title
learn = tabular_learner(dls, y_range=(0,1), layers=[100],
                        n_out=1, loss_func=nn.BCELoss()) # This is a learner in Fastai for which we will train our model. Here we're using binary cross entropy loss

In [ ]:
alpha = learn.lr_find().valley # Gets optimal learning rate. 

In [ ]:
learn.fit_one_cycle(10, alpha) # Fits model.

In [ ]:
#@title
test_dl = learn.dls.test_dl(df_embeds.loc[splits[1]]) # Dataloader used to test test data
predictions = learn.get_preds(dl=test_dl)
preds = np.array([i[0] > 0.5 for i in predictions[0]])
targets = df_embeds.loc[splits[1]]["reverse"]
corrects = preds == targets # ith element is true iff our prediction was correct
print("Neural Net Accuracy:", corrects.mean()) # Prints the accuracy